# Exercise 7) Learning and Planning

In this exercise, we will again investigate the inverted pendulum from the `gymnasium` environment. We want to check, which benefits the implementation of planning offers.

Please note that the parameter $n$ has a different meaning in the context of planning (number of planning steps per actual step) than in the context of n-step learning.

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import numpy as np
import gymnasium as gym
gym.logger.set_level(40)
plt.style.use('seaborn')

We will reuse the discretization routine from the previous exercise:

In [ ]:
d_T = 15
d_theta = 15
d_omega = 15


def discretize_state(states):
    limits = [1, 1, 8]
    nb_disc_intervals = [d_theta, d_theta, d_omega]

    # bring to value range [-1, 1]
    norm_states = [state / limit for state, limit in zip(states, limits)]
    interval_lengths = [2 / d for d in nb_disc_intervals]
    disc_state = [(norm_state + 1) // interval_length for norm_state,
                  interval_length in zip(norm_states, interval_lengths)]
    disc_state = [(state - 1) if state == d else state for state,
                  d in zip(disc_state, nb_disc_intervals)]  # ensure that disc_state < d

    return np.array(disc_state)


def continualize_action(disc_action):
    limit = 2
    interval_length = 2 / (d_T-1)
    norm_action = disc_action * interval_length
    cont_action = (norm_action - 1) * limit
    return np.array(cont_action).flatten()


## 1) Dyna-Q

Write a Dyna-Q algorithm to solve the inverted pendulum. Check the quality of the result for different number of episodes, number of steps per episode and number of planning steps per interaction.

Make sure that the total number of learning steps stays the same for different n, such that comparisons are fair:

$\text{episodes} \cdot \text{steps} \cdot (1+n) = \text{const.}$

Interesting metrics for a comparison could be e.g. the execution time (the tqdm loading bar shows execution time of loops, alternatively you can use the time.time() command to get the momentary system time in seconds) and training stability. 

![](DynaQ_Algo.png)

## Solution 1)

The solution code is given below.


In [ ]:
def interact(env, pi, state, deterministic, epsilon):
    """Interact with the environment to get to the next state.

    Args:
        env: The environment to interact with
        pi: The policy
        state: The state before interaction
        deterministic: Whether actions are chosen deterministically or eps-greedily
        epsilon: The probability for random interaction

    Returns:
        next_state: The (discretized) next state after interaction
        reward: The reward for the current interaction
        terminated: If the goal was reached
        action: The applied action
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return next_state, reward, terminated, action


def learn(action_values, pi, state, action, next_state, reward, gamma, alpha):
    """Learn from an interaction with the environment. This process is
    identical for momentary data and data sampled from the model.

    Args:
        action_values: The action-value estimates before the update step
        pi: The policy before the update step
        state: The state before interaction
        action: The chosen action
        next_state: The state after interaction
        reward: The reward for the interaction
        gamma: Discount factor
        alpha: Step size

    Returns:
        action_values: The updated action-value estimates
        pi: The updated policy
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return action_values, pi


def learn_planning_from_experience(action_values, pi, model, gamma, alpha):
    """Sample interactions from the environment and learn from them.

    Args:
        action_values: The action-value estimates before the update step
        pi: The policy before the update step
        model: The current model of past experiences
        gamma: Discount factor
        alpha: Step size

    Returns:
        action_values: The updated action-value estimates
        pi: The updated policy

    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return action_values, pi

In [ ]:
def pendulumDynaQ(alpha, gamma, epsilon, n, nb_episodes, nb_steps):

    env = gym.make('Pendulum-v1')
    env = env.unwrapped

    action_values = np.zeros([d_theta, d_theta, d_omega, d_T])
    pi = np.zeros([d_theta, d_theta, d_omega])

    model = {}

    cumulative_reward_history = [] # we can use this to figure out how well the learning worked
    pbar = tqdm(range(nb_episodes))

    for _ in pbar:
        
        rewards = [] # this time, this list is only for monitoring

        state, _ = env.reset() # initialize x_0
        state = tuple(discretize_state(state).astype(int))

        for _ in range(nb_steps):

            # YOUR CODE HERE
            raise NotImplementedError()
        
        pbar.set_description(f"Cumulative_reward {np.sum(rewards):6.0f}, Progress")
        cumulative_reward_history.append(np.sum(rewards))

    env.close()    
    return cumulative_reward_history, pi

Function to evaluate and render the measurement using the gym environment:

In [ ]:
def experiment(pi, nb_steps=300, render=False):
# Runs the inverted pendulum experiment using policy pi for nb_steps steps

    if render:
        env = gym.make('Pendulum-v1', render_mode="human")
    else:
        env = gym.make('Pendulum-v1')
    env = env.unwrapped

    state, _ = env.reset() # initialize x_0
    state = tuple(discretize_state(state).astype(int))

    for k in range(nb_steps):
        if render:
            env.render() # comment out for faster execution

        next_state, reward, terminated, action = interact(
            env, pi, state, deterministic=True, epsilon=0
        )

        state = next_state

        if terminated:
            break

    env.close()

Let's use nb_episodes = 5000, nb_steps = 500, n = 0 as a first try. This is effectively Q learning.

\begin{align}
5000 \cdot 500 \cdot (0+1) &= 2.5 \cdot 10^6
\end{align}

The resulting policy is satisfactory.



In [ ]:
# train
print("Run without planning")
no_planning_history, no_planning_pi = pendulumDynaQ(alpha=0.1, gamma=0.9, epsilon=0.1, n=0, nb_episodes=5000, nb_steps=500)

In [ ]:
# run and render the experiment
experiment(no_planning_pi, nb_steps=300, render=True)

Now let's try $n=9$ with the same nb_steps = 500:

\begin{align}
\text{nb\_episodes} &= \frac{2.5 \cdot 10^6}{500 \cdot (9+1)} = 500
\end{align}

The resulting policy also looks good.



In [ ]:
# train
print("Run with planning")
with_planning_history, with_planning_pi = pendulumDynaQ(alpha=0.1, gamma=0.9, epsilon=0.1, n=9, nb_episodes=500, nb_steps=500)

In [ ]:
# run and render the experiment
experiment(with_planning_pi, nb_steps=300, render=True)

Now lets compare the cumulative rewards:


In [ ]:
plt.plot(no_planning_history, label="no planning")
plt.plot(with_planning_history, label="planning from experience")
plt.xlabel("episode")
plt.ylabel(r"$\sum R$")
plt.legend()
plt.show()


The cumulative reward over the episodes both seems to have high variance.

- Do you have an idea why? Consider the initial state of the environment after resetting.

So why should we prefer the planning method?

- **Planning leads to the agent interacting less often with the "real" environment, such that in the end less interaction time is needed.** To motivate this, imagine that you interact with a real test bench instead of a simulation and that every interaction with the test bench is costly either in terms of time or in terms of money. When compute costs are lower than interaction costs, it makes sense to revisit already gathered data instead of perfoming real interactions.

## 2) Simulation-based planning

Although it can be useful for small state spaces, building a system model by storing large amounts of state transitions like in task (1) is rarely feasible in engineering. As engineers, we are capable of a more efficient way of system modeling that we can utilize here: differential equations.

Using a state-space model allows to efficiently integrate existing pre-knowledge into the Dyna-Q algorithm we already used. To do so, write a class `pendulum_model` that implements a model of the pendulum. This class should work similar to `gymnasium`: it should at least have a `step` and a `reset` method. In the step method, make use of forward Euler integration to simulate the system dynamics. In the reset method, allow to pass an optional initial state to the model, such that we can easily compare model and environment. If no initial state is passed to the `reset` function, a random initial state should be determined.

Integrate this model into a Dyna-Q algorithm.

Model of the pendulum in differential-equation form for change of the angular frequency $\omega$ and the angle $\theta$ depending on the torque $T_\mathrm{u}$:

\begin{align*}
\dot{\omega} &= -\frac{3 g}{2 l} \text{sin}(\theta +\pi) + \frac{1}{J} T_\mathrm{u}
\\
\dot{\theta} &= \omega
\end{align*}



Parameters (gravity constant $g$, mass $m$, length  $l$ and intertia $J$ of the pendulum):

\begin{align*}
g&=10 \, \frac{\text{m}}{\text{s}^2}
\\
m&=1 \, \text{kg}
\\
l&=1 \, \text{m}
\\
J&=\frac{1}{3} m l^2
\end{align*}

Forward Euler integration:

\begin{align*}
\dot{x}(k T_S) \approx \frac{x[k+1] - x[k]}{T_S}
\end{align*}

with sampling time $T_S = 0.05 \, \text{s}$

Reward function:

\begin{align*}
r_{k+1} = -(\theta^2[k] + 0.1 \, \text{s}^2 \cdot \omega^2[k] + 0.001 \frac{1}{(\text{N}\text{m})^2} \cdot T_\mathrm{u}^2[k])
\end{align*}

Limitations of state and action space:
\begin{align*}
\theta &\in [-\pi, \pi]
\\
\omega &\in [-8  \, \frac{1}{\text{s}}, 8  \, \frac{1}{\text{s}}]
\\
T_\mathrm{u} &\in [-2 \, \text{N}\text{m}, 2 \, \text{N}\text{m}]
\end{align*}

And of course input and output space:
\begin{align*}
\text{action}&=T_\mathrm{u}
\\
\text{state}&=
\begin{bmatrix}
\text{cos}(\theta)\\
\text{sin}(\theta)\\
\omega
\end{bmatrix}
\end{align*}

## Solution 2)

Model-based planning does not necessarily run faster than experience-based planning. However, experience-based planning fails to cover the whole state space especially in the earlier episodes when there are too few experiences. On the other hand, model-based planning can, of course, only be performed if a state-space model with accurate parametrization is available. In order to overcome parametric deviations between state-space model and environment, one could even use the measurements from the actual environment in order to identify the parameters of the model during runtime.

In [ ]:
class pendulum_model:
    def __init__(self, dt=0.05, m=1, g=10, l=1):
        """
        Initializes the Pendulum Model.

        Args:
            dt (float): Sampling time in seconds.
            m (float): Mass in kilograms.
            g (float): Gravity in meters per second squared.
            l (float): Length in meters.
        """

        # YOUR CODE HERE
        raise NotImplementedError()

    def reset(self, state=None):
        """
        Resets the pendulum's state.

        Args:
            state (array): Initial state (theta, omega) in radians and radians per second.

        Returns:
            state (array): Current state after reset.
        """

        # YOUR CODE HERE
        raise NotImplementedError()

        return state

    def step(self, T_u):
        """
        Takes a step in the simulation.

        Args:
            T_u (float): Control torque applied.

        Returns:
            state (array): Current state after the step.
            reward (float): Reward associated with the step.
        """

        # YOUR CODE HERE
        raise NotImplementedError()

        return state, reward

    def angle_normalize(self, theta):
        """
        Normalizes the angle.

        Args:
            theta (float): Angle in radians.

        Returns:
            float: Normalized angle in radians.
        """

        # usage of this helper function is optional
        return (((theta+np.pi) % (2*np.pi)) - np.pi)

The following cell is for debugging of the `pendulum_model` class. 

In [ ]:
env = gym.make('Pendulum-v1')
# removes a builtin time limit of k_T = 200, we want to set the time limit ourselves
env = env.unwrapped

model = pendulum_model()

state, _ = env.reset()

m_state = model.reset(state)  # model is set to state of env

nb_episodes = 10000

for _ in range(nb_episodes):

    action = env.action_space.sample()

    state, reward, terminated, _, _ = env.step(action)  # take action on env
    m_state, m_reward = model.step(action)  # take the same action on model

env.close()


Using $-\mathrm{sin}(\theta)$ instead of $\mathrm{sin}(\theta +\pi)$ makes no difference when assuming analytical precision, but due to numeric errors these formulations will still yield different results in numpy, mainly because $\pi$ is represented with finite (float) precision. In order to yield the same numbers as in `gymnasium`, we will still make use of the (more cumbersome) $\mathrm{sin}(\theta +\pi)$.

Write a function for the Dyna-Q algorithm, which uses the model we defined above: 

In [ ]:
# You may reuse interact and learn from the previous task

def learn_planning_from_model(action_values, pi, model, gamma, alpha):
    """Sample interactions from the environment and learn from them.

    Args:
        action_values: The action-value estimates before the update step
        pi: The policy before the update step
        model: The current model of the environment
        gamma: Discount factor
        alpha: Step size

    Returns:
        action_values: The updated action-value estimates
        pi: The updated policy

    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return action_values, pi

In [ ]:
def pendulumModelDynaQ(alpha, gamma, epsilon, n, nb_episodes, nb_steps, m=1, g=10, l=1):

    env = gym.make('Pendulum-v1')  # , render_mode="human"
    env = env.unwrapped
    model = pendulum_model(m=m, g=g, l=l)

    action_values = np.zeros([d_theta, d_theta, d_omega, d_T])
    pi = np.zeros([d_theta, d_theta, d_omega])

    # we can use this to figure out how well the learning worked
    cumulative_reward_history = []

    pbar = tqdm(range(nb_episodes), position=0, leave=True)

    for _ in pbar:
        rewards = [] # this time, this list is only for monitoring
        
        # YOUR CODE HERE
        raise NotImplementedError()

        pbar.set_description(
            f"Cumulative_reward {np.sum(rewards):6.0f}, Progress")

        cumulative_reward_history.append(np.sum(rewards))

    env.close()

    return cumulative_reward_history, pi


Use the following cell to compare the learing from experience from 1) to the learning using the defined model: (Beware, nb_steps = 10000 can take some time)


In [ ]:
# train both setups once
print("Run with planning from experience")
exp_planning_history, exp_planning_pi = pendulumDynaQ(alpha=0.1, gamma=0.9, epsilon=0.1, n=19, nb_episodes=500, nb_steps=500)

In [ ]:
print("Run with planning from model")
model_planning_history, model_planning_pi = pendulumModelDynaQ(alpha=0.1, gamma=0.9, epsilon=0.1, n=19, nb_episodes=500, nb_steps=500)

In [ ]:
plt.plot(exp_planning_history, label="planning from experience")
plt.plot(model_planning_history, label="planning from model")
plt.xlabel("episode")
plt.ylabel(r"$\sum R$")
plt.legend(loc="upper left")
plt.show()

Use the following cell to execute the policy we got using the model:

In [ ]:
experiment(model_planning_pi, nb_steps=300, render=True)

Now we change the model parameters (e.g. $g$, $m$, $l$) so that our model differs from the "real world" (the gym environment).
What do you observe? You can also change the parameters to other values yourself.

The Following learning curve results from a parameter change using $g =20 \, \frac{\text{m}}{\text{s}^2}, m = 5 \, \text{kg and } l = 2 \, \text{m}$:

In [ ]:
wrong_model_history, wrong_model_pi = pendulumModelDynaQ(
    alpha=0.1, gamma=0.9, epsilon=0.1, n=19, nb_episodes=500, nb_steps=500, m=5, g=20, l=2
)

In [ ]:
plt.plot(exp_planning_history, label="planning from experience")
plt.plot(wrong_model_history, label="planning from poor model")
plt.xlabel("episode")
plt.ylabel(r"$\sum R$")
plt.legend(loc="upper left")
plt.show()

By changing the parameters our model differs from the "real world". Depending on the amount of difference, the learing curve looks worse than the one with the correct values. 
The experiment result is also depending on the random starting position. 
Depending on the parameter difference, the experiment can not be executed successfully any more. 

**In conclusion**: A good model can outperform experience gathered from the environment, but a poor model choice can also hinder the training process.